In [1]:
!pip install git+https://github.com/pytorch/tnt.git@master

  Cloning https://github.com/pytorch/tnt.git (to revision master) to c:\users\soheil~1\appdata\local\temp\pip-req-build-l80p5668
  Created wheel for torchnet: filename=torchnet-0.0.5.1-py3-none-any.whl size=31207 sha256=5f62040f6f6884a1b4e0d723d05758cb52a869fb45532f9f5a990b1779d77226
  Stored in directory: C:\Users\SOHEIL~1\AppData\Local\Temp\pip-ephem-wheel-cache-z7q9xskg\wheels\e1\d4\7a\e9a5ec0d0dcd304bfa84d2cbb8a7507a5e728ce154981dc87d
Successfully built torchnet


  Running command git clone -q https://github.com/pytorch/tnt.git 'C:\Users\SOHEIL~1\AppData\Local\Temp\pip-req-build-l80p5668'


In [2]:
!pip install torchvision

In [1]:
import torch
from torchvision import transforms
from torch.autograd import Variable

In [2]:
import os
import numpy as np
import cv2

save_img_path = "E:/my_courses/project/pykinect2/PyKinect2/examples/body_skeleton_image/"


In [86]:
def body_to_image(body_seq,index1=0,index2=0):#gets array with len 42 and each element is array of size 15 
        
        seq_lines = np.array(body_seq)
        X = np.array(seq_lines[:,:,0])
        Y = np.array(seq_lines[:,:,1])
        Z = np.array(seq_lines[:,:,2])

        X = X.reshape(X.reshape(15,-1).shape[1],15).transpose()
        X = X[8] - X # DISTANCE between torso and others
        X = np.delete(X, 1, 0) #delete torso 
        r = (255*(X-(X.min()))/((X.max())-(X.min()))).astype(int) # Normalization and put into to 0-255 range
        
        Y = Y.reshape(Y.reshape(15,-1).shape[1],15).transpose()
        Y = Y[8] - Y
        Y = np.delete(Y, 1, 0) #delete torso
        g = (255*(Y-(Y.min()))/((Y.max())-(Y.min()))).astype(int)
        
        Z = Z.reshape(Z.reshape(15,-1).shape[1],15).transpose()
        Z = Z[8] - Z  
        Z = np.delete(Z, 1, 0) #delete torso
        b = (255*(Z-(Z.min()))/((Z.max())-(Z.min()))).astype(int)
        
        image = cv2.merge((r,g,b))
        cv2.imwrite(save_img_path+str(index1)+'.jpg',image)
        print('body to image',image.shape)
        return image
        #print(img_name)
    

In [4]:
classes = ['Bend', 'Hand Clap', 'Walk', 'Sit down', 'Stand up']

In [5]:
model_path = 'E:/my_courses/project/final_files/model/'
model=torch.load(model_path+'Mymodel_003_repair_dataset.pth',map_location=torch.device('cpu'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
from PIL import Image

In [7]:
def test_model(img):
#     to_pil = transforms.ToPILImage()
#     image = to_pil(img)
    image = Image.fromarray(img.astype('uint8'), 'RGB')
    test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input1 = Variable(image_tensor)
    input1 = input1.to('cpu')
    output = model(input1)
    index = output.data.cpu().numpy().argmax()
    
    return str(classes[index])

In [31]:
global skel

In [64]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
import numpy as np 
import ctypes
import _ctypes
import pygame
import sys

if sys.hexversion >= 0x03000000:
    import _thread as thread
else:
    import thread

# colors for drawing different bodies 
SKELETON_COLORS = [pygame.color.THECOLORS["red"], 
                  pygame.color.THECOLORS["blue"], 
                  pygame.color.THECOLORS["green"], 
                  pygame.color.THECOLORS["orange"], 
                  pygame.color.THECOLORS["purple"], 
                  pygame.color.THECOLORS["yellow"], 
                  pygame.color.THECOLORS["violet"]]


class BodyGameRuntime(object):
    def __init__(self):
        pygame.init()

        # Used to manage how fast the screen updates
        self._clock = pygame.time.Clock()

        # Set the width and height of the screen [width, height]
        self._infoObject = pygame.display.Info()
        self._screen = pygame.display.set_mode((self._infoObject.current_w >> 1, self._infoObject.current_h >> 1), 
                                               pygame.HWSURFACE|pygame.DOUBLEBUF|pygame.RESIZABLE, 32)

        pygame.display.set_caption("Kinect for Windows v2 Body Game")

        # Loop until the user clicks the close button.
        self._done = False
        self._depth = []
        # Used to manage how fast the screen updates
        self._clock = pygame.time.Clock()

        # Kinect runtime object, we want only color and body and depth frames 
        self._kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth | 
                                                       PyKinectV2.FrameSourceTypes_Color | 
                                                       PyKinectV2.FrameSourceTypes_Body)

        # back buffer surface for getting Kinect color frames, 32bit color, width and height equal to the Kinect color frame size
        self._frame_surface = pygame.Surface((self._kinect.color_frame_desc.Width, self._kinect.color_frame_desc.Height), 0, 32)
        #print(self._frame_surface.get_width())
        # here we will store skeleton data 
        self._bodies = None
        
        self.Body_keypoints = [] #store all frames keypoints
        self.One_frame_points = np.zeros((25,3)) #store a frame keypoints
        self.condition_on_points = [PyKinectV2.JointType_Head, PyKinectV2.JointType_Neck,PyKinectV2.JointType_ShoulderRight,
                                    PyKinectV2.JointType_ShoulderLeft,PyKinectV2.JointType_SpineMid,PyKinectV2.JointType_ElbowRight
                                    ,PyKinectV2.JointType_HipRight,PyKinectV2.JointType_HipLeft,PyKinectV2.JointType_HandRight,
                                    PyKinectV2.JointType_ElbowLeft,
                                    PyKinectV2.JointType_HandLeft,PyKinectV2.JointType_KneeRight,
                                    PyKinectV2.JointType_FootRight,PyKinectV2.JointType_KneeLeft,
                                    PyKinectV2.JointType_FootLeft] #check if joint0 or joint1 is in this 15 keypoints
        self.joint_depth = []
        
        self.first = True
        self.f = 0
        self.e = 42
        
        self.text = ""

    
    def draw_body_bone(self, joints, jointPoints, color, joint0, joint1):
        
        joint0State = joints[joint0].TrackingState;
        joint1State = joints[joint1].TrackingState;

        # both joints are not tracked
        if (joint0State == PyKinectV2.TrackingState_NotTracked) or (joint1State == PyKinectV2.TrackingState_NotTracked): 
            return

        # both joints are not *really* tracked
        if (joint0State == PyKinectV2.TrackingState_Inferred) and (joint1State == PyKinectV2.TrackingState_Inferred):
            return

        # ok, at least one is good 
        if (joint0 in self.condition_on_points) and (self.One_frame_points[joint0].all() == 0):
            x = int(self.joint_depth[joint0].x)
            y = int(self.joint_depth[joint0].y)
            z = int(self._depth[ y * 512 + x ] ) ## Its in Millimeters
            self.One_frame_points[joint0] = ([x,y,z])
            
        if (joint1 in self.condition_on_points) and (self.One_frame_points[joint1].all() == 0):
            x = int(self.joint_depth[joint1].x)
            y = int(self.joint_depth[joint1].y)
            z = int(self._depth[ y * 512 + x ] ) ## Its in Millimeters
            self.One_frame_points[joint1] = ([x,y,z])
            if joint1 == PyKinectV2.JointType_FootLeft:
                self.Body_keypoints.append(self.One_frame_points[[3,2,8,9,11,4,5,7,1,16,17,19,12,13,15]])
                self.One_frame_points = np.zeros((25,3)) 
                print('Body size:',len(self.Body_keypoints))
                # model ♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠
#                 if len(self.Body_keypoints) >= 42:
#                     image = body_to_image(self.Body_keypoints[-42:],self.f,self.e)
#                     #self.Body_keypoints = self.Body_keypoints[2:]

#                     self.text = test_model(image)
#                     self.f += 42
#                     self.e += 42
#                 else:
#                     if self.first == True:
#                         self.text = "Nothing"
#                         self.first = False
                #model ♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠♠
        
        start = (jointPoints[joint0].x, jointPoints[joint0].y)
        end = (jointPoints[joint1].x, jointPoints[joint1].y)

        try:
            pygame.draw.line(self._frame_surface, color, start, end, 8)
        except: # need to catch it due to possible invalid positions (with inf)
            pass
        

    def draw_body(self, joints, jointPoints, color):
        # Torso
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_Head, PyKinectV2.JointType_Neck);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_Neck, PyKinectV2.JointType_SpineShoulder);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineShoulder, PyKinectV2.JointType_SpineMid);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineMid, PyKinectV2.JointType_SpineBase);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineShoulder, PyKinectV2.JointType_ShoulderRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineShoulder, PyKinectV2.JointType_ShoulderLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineBase, PyKinectV2.JointType_HipRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_SpineBase, PyKinectV2.JointType_HipLeft);
    
        # Right Arm    
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_ShoulderRight, PyKinectV2.JointType_ElbowRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_ElbowRight, PyKinectV2.JointType_WristRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_WristRight, PyKinectV2.JointType_HandRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_HandRight, PyKinectV2.JointType_HandTipRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_WristRight, PyKinectV2.JointType_ThumbRight);

        # Left Arm
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_ShoulderLeft, PyKinectV2.JointType_ElbowLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_ElbowLeft, PyKinectV2.JointType_WristLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_WristLeft, PyKinectV2.JointType_HandLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_HandLeft, PyKinectV2.JointType_HandTipLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_WristLeft, PyKinectV2.JointType_ThumbLeft);

        # Right Leg
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_HipRight, PyKinectV2.JointType_KneeRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_KneeRight, PyKinectV2.JointType_AnkleRight);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_AnkleRight, PyKinectV2.JointType_FootRight);

        # Left Leg
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_HipLeft, PyKinectV2.JointType_KneeLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_KneeLeft, PyKinectV2.JointType_AnkleLeft);
        self.draw_body_bone(joints, jointPoints, color, PyKinectV2.JointType_AnkleLeft, PyKinectV2.JointType_FootLeft);


    def draw_color_frame(self, frame, target_surface):
        target_surface.lock()
        address = self._kinect.surface_as_array(target_surface.get_buffer())
        ctypes.memmove(address, frame.ctypes.data, frame.size)
        del address
        target_surface.unlock()

    def run(self):
        global skel
        pygame.time.delay(2000)
        # timer
        pygame.time.set_timer(pygame.USEREVENT, 1000)
        counter = 0
        image_indx = 10
        #limit = 1000
        save_img = False
        
        # -------- Main Program Loop -----------
        while not self._done:
            # --- Main event loop
            for event in pygame.event.get(): # User did something
                if event.type == pygame.QUIT: # If user clicked close
                    self._done = True # Flag that we are done so we exit this loop

                elif event.type == pygame.VIDEORESIZE: # window resized
                    self._screen = pygame.display.set_mode(event.dict['size'], 
                                               pygame.HWSURFACE|pygame.DOUBLEBUF|pygame.RESIZABLE, 32)

                #timer
                elif event.type == pygame.USEREVENT: 
                    counter += 1
                    #print(1)
                    if counter == 3:
                        self.text = self.text+":running again"
                    if counter == 5:
                        #print(2)
                        counter = 0 
                        image = body_to_image(self.Body_keypoints,image_indx+1)
                        skel = image
                        image_indx += 1
                        self.text = test_model(image) + ":end"   
                        print(self.text)
                        self.Body_keypoints = []
                        

                #timer
                
            if self._kinect.has_new_color_frame():
                frame = self._kinect.get_last_color_frame()
                self.draw_color_frame(frame, self._frame_surface)
                frame = None
            
            #depth
            if self._kinect.has_new_depth_frame():
                self._depth = self._kinect.get_last_depth_frame()
                
            #depth
            
            # --- Cool! We have a body frame, so can get skeletons
            if self._kinect.has_new_body_frame(): 
                self._bodies = self._kinect.get_last_body_frame()

            # --- draw skeletons to _frame_surface
            if self._bodies is not None: 
                for i in range(0, self._kinect.max_body_count):
                    body = self._bodies.bodies[i]
                    if not body.is_tracked: 
                        continue 
                    
                    joints = body.joints 
                    # convert joint coordinates to color space 
                    joint_points = self._kinect.body_joints_to_color_space(joints)
                    self.joint_depth = self._kinect.body_joints_to_depth_space(joints)
                    

                    self.draw_body(joints, joint_points, SKELETON_COLORS[i])
                    

                    
            # --- copy back buffer surface pixels to the screen, resize it if needed and keep aspect ratio
            # --- (screen size may be different from Kinect's color frame size) 
            h_to_w = float(self._frame_surface.get_height()) / self._frame_surface.get_width()
            target_height = int(h_to_w * self._screen.get_width())
            surface_to_draw = pygame.transform.scale(self._frame_surface, (self._screen.get_width(), target_height));
            self._screen.blit(surface_to_draw, (0,0))
            surface_to_draw = None
            
            #text
            font = pygame.font.Font('freesansbold.ttf', 50)
            text = font.render('Action:'+self.text, True, pygame.color.THECOLORS["red"], pygame.color.THECOLORS["blue"])
            self._screen.blit(text,(0,0))
            #text
            
            pygame.display.update()

            # --- Go ahead and update the screen with what we've drawn.
            pygame.display.flip()

            # --- Limit to 60 frames per second
            self._clock.tick(60)

        # Close our Kinect sensor, close the window and quit.
        a = self.Body_keypoints
        self._kinect.close()
        pygame.quit()


__main__ = "Kinect v2 Body Game"
game = BodyGameRuntime();
game.run();


Body size: 1
Body size: 2
Body size: 3
Body size: 4
Body size: 5
Body size: 6
Body size: 7
Body size: 8
Body size: 9
Body size: 10
Body size: 11
Body size: 12
Body size: 13
Body size: 14
Body size: 15
Body size: 16
Body size: 17
Body size: 18
Body size: 19
Body size: 20
Body size: 21
Body size: 22
Body size: 23
Body size: 24
Body size: 25
Body size: 26
Body size: 27
Body size: 28
Body size: 29
Body size: 30
Body size: 31
Body size: 32
Body size: 33
Body size: 34
Body size: 35
Body size: 36
Body size: 37
Body size: 38
Body size: 39
Body size: 40
Body size: 41
Body size: 42
Body size: 43
Body size: 44
Body size: 45
Body size: 46
Body size: 47
Body size: 48
Body size: 49
Body size: 50
Body size: 51
Body size: 52
Body size: 53
Body size: 54
Body size: 55
Body size: 56
Body size: 57
Body size: 58
Body size: 59
Body size: 60
Body size: 61
Body size: 62
Body size: 63
Body size: 64
Body size: 65
Body size: 66
Body size: 67
Body size: 68
Body size: 69
Body size: 70
Body size: 71
Body size: 72
B

Body size: 12
Body size: 13
Body size: 14
Body size: 15
Body size: 16
Body size: 17
Body size: 18
Body size: 19
Body size: 20
Body size: 21
Body size: 22
Body size: 23
Body size: 24
Body size: 25
Body size: 26
Body size: 27
Body size: 28
Body size: 29
Body size: 30
Body size: 31
Body size: 32
Body size: 33
Body size: 34
Body size: 35
Body size: 36
Body size: 37
Body size: 38
Body size: 39
Body size: 40
Body size: 41
Body size: 42
Body size: 43
Body size: 44
Body size: 45
Body size: 46
Body size: 47
Body size: 48
Body size: 49
Body size: 50
Body size: 51
Body size: 52
Body size: 53
Body size: 54
Body size: 55
Body size: 56
Body size: 57
Body size: 58
Body size: 59
Body size: 60
Body size: 61
Body size: 62
Body size: 63
Body size: 64
Body size: 65
Body size: 66
Body size: 67
Body size: 68
Body size: 69
Body size: 70
Body size: 71
Body size: 72
Body size: 73
Body size: 74
Body size: 75
Body size: 76
Body size: 77
Body size: 78
Body size: 79
Body size: 80
Body size: 81
Body size: 82
Body s

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [65]:
game._kinect.close()
pygame.quit()

In [68]:
import gc
gc.collect()

0

## Body skeleton used by KARD dataset

In [27]:
# Head = 3  
# Neck = 2  
# Right_Shoulder = 8
# Right_Elbow = 9
# Right_Hand = 11
# Left_Shoulder = 4
# Left_Elbow = 5
# Left_Hand = 7
# Torso = 1
# Right_Hip = 16
# Right_Knee = 17
# Right_Foot = 19 
# Left_Hip = 12
# Left_Knee = 13
# Left_Foot = 15